# Evaluate ensemble members

The goal of this script is to identify ensemble members that meet specific ecological criteria

In [79]:
import pandas as pd
import numpy as np
import os
import esm_tools
pd.set_option('display.max_rows', 500) 
import shutil

In [70]:
# Path to where the prior ensemble run parameter files are stored
path_to_ensemble_param_files_root = '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/'

In [15]:
def get_most_recent_file_in_dir(folder_path):

    # Get a list of all files in the folder
    files = os.listdir(folder_path)

    # Filter out only files (excluding subdirectories)
    files = [file for file in files if os.path.isfile(os.path.join(folder_path, file))]

    # Sort the files by creation time (most recent first)
    files.sort(key=lambda x: os.path.getctime(os.path.join(folder_path, x)), reverse=True)

    # Check if there are any files in the folder
    if files:
        most_recent_file = files[0]
        print(f"The most recently created file is: {most_recent_file}")
        return most_recent_file
    else:
        print("The folder is empty.")
        


## Set up which files are being aggregated and evaluated

In [24]:
case_name_prefix = 'ca_5pfts_20cases_4320inst_101223_'
case_name_suffix = '_-17e2acb6a_FATES-8a054a12'
case_numbers = [3,4,5,6,7,8,12,13,15,16,17,18,19,20]

# Path to put any processed output
processed_output_root = '/glade/scratch/adamhb/processed_output'

case_tags = [str(case_num).rjust(2, '0') for case_num in case_numbers]
print("Case Tags:")
print(case_tags)
print("\n")
folder_names = [case_name_prefix + case_tag + case_name_suffix for case_tag in case_tags]
folder_paths = [os.path.join(processed_output_root,folder_name) for folder_name in folder_names]

print("Folders:")
print(folder_paths)
print("\n")

print("Files")
[get_most_recent_file_in_dir(folder_path) for folder_path in folder_paths]
print("\n")

print("Full file paths:")
files = [get_most_recent_file_in_dir(folder_path) for folder_path in folder_paths]
full_file_paths = [os.path.join(folder_paths[i],files[i]) for i in range(len(files))]
print(full_file_paths)

Case Tags:
['03', '04', '05', '06', '07', '08', '12', '13', '15', '16', '17', '18', '19', '20']


Folders:
['/glade/scratch/adamhb/processed_output/ca_5pfts_20cases_4320inst_101223_03_-17e2acb6a_FATES-8a054a12', '/glade/scratch/adamhb/processed_output/ca_5pfts_20cases_4320inst_101223_04_-17e2acb6a_FATES-8a054a12', '/glade/scratch/adamhb/processed_output/ca_5pfts_20cases_4320inst_101223_05_-17e2acb6a_FATES-8a054a12', '/glade/scratch/adamhb/processed_output/ca_5pfts_20cases_4320inst_101223_06_-17e2acb6a_FATES-8a054a12', '/glade/scratch/adamhb/processed_output/ca_5pfts_20cases_4320inst_101223_07_-17e2acb6a_FATES-8a054a12', '/glade/scratch/adamhb/processed_output/ca_5pfts_20cases_4320inst_101223_08_-17e2acb6a_FATES-8a054a12', '/glade/scratch/adamhb/processed_output/ca_5pfts_20cases_4320inst_101223_12_-17e2acb6a_FATES-8a054a12', '/glade/scratch/adamhb/processed_output/ca_5pfts_20cases_4320inst_101223_13_-17e2acb6a_FATES-8a054a12', '/glade/scratch/adamhb/processed_output/ca_5pfts_20cases_432

## Get all data into on dataframe

In [96]:
df = pd.DataFrame()
i = 0
for f in full_file_paths:
    df_tmp = pd.read_csv(f)
    df_tmp = df_tmp.drop(df_tmp.columns[0], axis=1)
    df_tmp['case_tag'] = case_tags[i]
    df = pd.concat([df,df_tmp],axis = 0)
    i = i + 1

In [97]:
df.to_csv(processed_output_root + "/" + "all_member_output_101223.csv")

In [37]:
cols = ["case_tag","inst","BA","BA_pine","BA_cedar","BA_fir","BA_shrub","BA_oak",
 "AGB","TreeStemD","ResproutD_oak","ResproutD_shrub","ShannonE","NPP","FailedPFTs",
"Pct_shrub_cover_canopy","Pct_shrub_cover","Combustible_fuel"]

In [40]:
df = df[cols]

In [94]:
df.shape

(3024, 18)

## Filter to ensemble members that meet ecological criteria

CZ2 Pre-Euro-American Management--criteria for analyzing fire regimes

1. There are 0 failing pfts (all pfts have above at least 0.1 m2 ha-1 of basal area)
2. Shrub percent cover is > 1% and < 50%

In [68]:
CZ2_analyze_fire_regime_filter = (df['FailedPFTs'] == 0) & (df['Pct_shrub_cover'] > 0.01) & (df['Pct_shrub_cover_canopy'] < 0.5)
CZ2_analyze_fire_regime = df.loc[CZ2_analyze_fire_regime_filter].sort_values("ShannonE",ascending = False)
CZ2_analyze_fire_regime.to_csv(os.path.join(processed_output_root,"successfull_members_ca_5pfts_20cases_4320inst_101223_XX_-17e2acb6a_FATES-8a054a12.csv"))


#Take just the top 108 members to run a 3-node simulation to equilibrium
CZ2_analyze_fire_regime = CZ2_analyze_fire_regime[:108]

CZ2_analyze_fire_regime.shape

(108, 18)

In [82]:
CZ2_analyze_fire_regime

,case_tag,inst,BA,BA_pine,BA_cedar,BA_fir,BA_shrub,BA_oak,AGB,TreeStemD,ResproutD_oak,ResproutD_shrub,ShannonE,NPP,FailedPFTs,Pct_shrub_cover_canopy,Pct_shrub_cover,Combustible_fuel
35,04,36,28.954632,6.161109,5.381263,2.644943,10.754788,4.012528,4.354155,141.845644,54.376423,0.000000,0.933477,0.754083,0,0.007918,0.507289,0.595101
133,13,134,16.373880,3.121986,4.088163,0.793111,3.470524,4.900097,3.628098,167.905092,196.862180,0.000000,0.931341,0.726551,0,0.045620,0.131540,0.524947
120,03,121,26.095877,2.165925,8.841488,4.292324,7.420950,3.375191,5.471643,214.657933,27.892290,0.000000,0.927211,0.544646,0,0.097570,0.289345,0.589385
91,05,92,21.963913,2.698711,7.426689,3.862811,1.510597,6.465104,6.980218,185.799152,38.220146,0.000000,0.915856,0.683593,0,0.037777,0.081788,0.483851
4,04,5,15.671631,1.184783,2.222071,4.308748,2.097477,5.858551,3.444905,318.851434,30.531569,0.000000,0.909756,0.612420,0,0.002431,0.120956,0.531337
59,07,60,18.780384,1.482514,2.876525,2.360527,7.344179,4.716641,3.146393,119.238123,73.938110,0.033478,0.908803,0.709801,0,0.235475,0.491884,0.957490
20,15,21,12.094015,1.393123,1.110006,2.229052,1.592901,5.768933,2.730853,188.798588,90.677980,0.000203,0.869828,0.490590,0,0.093193,0.171676,0.376983
146,20,147,20.544651,8.345717,3.335245,0.551904,4.908621,3.403164,4.380298,162.375644,29.358751,0.027759,0.868701,0.661550,0,0.154757,0.199823,0.450695
111,17,112,33.504627,6.967893,9.325161,2.854433,12.946913,1.410228,5.515543,265.341178,17.696530,0.000000,0.865603,0.834924,0,0.272900,0.465976,0.656638
39,07,40,15.264673,5.497664,3.821568,0.162287,2.759563,3.023590,2.743626,202.919878,109.151290,0.000000,0.865360,0.642794,0,0.075405,0.122316,0.599947


## Prepare passing parameterizations for next simulation

In [85]:
new_case_tags

NameError: name 'new_case_tags' is not defined

In [92]:
def aggregate_passing_paramsets(pass_case_tags,passing_inst_tags,n_new_cases,n_inst_per_case,new_subdirs_prefix,
                                path_to_ensemble_param_files_root,old_param_subdir_prefix,new_param_file_base_name):
    
    
    for n in range(n_new_cases):
        new_param_subdir_suffix = str(n+1).rjust(2, '0')
        
        new_param_subdir = os.path.join(path_to_ensemble_param_files_root,new_subdirs_prefix) + "_" + new_param_subdir_suffix
        os.makedirs(new_param_subdir,exist_ok=True)
    
    tmp_inst = 0
    for case_tag,inst_tag in zip(pass_case_tags,passing_inst_tags):
        
        tmp_inst = tmp_inst + 1
        
        param_files_subdir = old_param_subdir_prefix + "_" + case_tag
        
        # Get tag
        inst_tag_with_leading_zeroes = str(inst_tag).rjust(4, '0')
    
        # Get param file with inst tag
        ref_nc_file = esm_tools.find_files_with_substring(directory=os.path.join(path_to_ensemble_param_files_root,
                                                                       param_files_subdir),
                                                substring=inst_tag_with_leading_zeroes)

        # Get full path of originl param file
        ref_nc_file_full_path = os.path.join(path_to_ensemble_param_files_root,param_files_subdir,ref_nc_file[0])

        print("originl_file:",ref_nc_file_full_path)

        
        #Copy to new folder
        os.makedirs("tmp",exist_ok=True)
        
        new_tag = str(tmp_inst).rjust(4, '0')
    
        new_name = new_param_file_base_name + "_" + new_tag + ".nc"

        dst_file = os.path.join("tmp",new_name)
        
        print("tmp file:",dst_file)
        print("\n")

        shutil.copy(ref_nc_file_full_path,dst_file)
        
        
        # allocate across new cases
        new_case_tags = [str(case_tag+1).rjust(2, '0') for case_tag in range(n_new_cases)]
      
        
    for i,new_case_tag in enumerate(new_case_tags):
            
            new_param_subdir = os.path.join(path_to_ensemble_param_files_root,new_subdirs_prefix) + "_" + new_case_tag
            
            tmp_inst = i * n_inst_per_case
            
            for i in range(n_inst_per_case):
                
                tmp_inst = tmp_inst + 1
                
                tmp_inst_tag = str(tmp_inst).rjust(4, '0')
                
                 # Get param file with inst tag
                ref_nc_file = esm_tools.find_files_with_substring(directory="tmp",
                                                substring=tmp_inst_tag)

                # Get full path of originl param file
                ref_nc_file_full_path = os.path.join("tmp",ref_nc_file[0])
                
                print("tmp_file:",ref_nc_file_full_path)
                
                new_tag = str(i + 1).rjust(4, '0')
                new_name = new_param_file_base_name + "_" + new_tag + ".nc"
                dst_file = os.path.join(path_to_ensemble_param_files_root,new_param_subdir,new_name)
                
                shutil.copy(ref_nc_file_full_path,dst_file)
                
                print("destination:",dst_file)    

In [93]:
aggregate_passing_paramsets(pass_case_tags = CZ2_analyze_fire_regime['case_tag'],
                            passing_inst_tags = CZ2_analyze_fire_regime['inst'],
                            n_new_cases = 3,
                            n_inst_per_case = 36,
                            new_subdirs_prefix = "CZ2_equilibrium_101723",
                            path_to_ensemble_param_files_root = path_to_ensemble_param_files_root,
                            old_param_subdir_prefix = 'ca_5pfts_20cases_4320inst_101223',
                            new_param_file_base_name = 'CZ2_equilibrium_101723')

originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_04/ca_5pfts_100523_0036.nc
tmp file: tmp/CZ2_equilibrium_101723_0001.nc


originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_13/ca_5pfts_100523_0134.nc
tmp file: tmp/CZ2_equilibrium_101723_0002.nc


originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_03/ca_5pfts_100523_0121.nc
tmp file: tmp/CZ2_equilibrium_101723_0003.nc


originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_05/ca_5pfts_100523_0092.nc
tmp file: tmp/CZ2_equilibrium_101723_0004.nc


originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_04/ca_5pfts_100523_0005.nc
tmp file: tmp/CZ2_equilibrium_101723_0005.nc


originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_07/ca_5pfts_100523